# Explanation

This code implements a simple trading strategy based on a rolling maximum and minimum of the stock prices. It generates buy and sell signals based on the relationship between the current price and the rolling max/min. The buy_stock function simulates the actual trading process, keeping track of the available money, inventory, and transaction history. Finally, it returns the performance metrics of the trading strategy.

In [2]:
from utils import *
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from collections import deque
import random

# Configure Modeling Parameters and Fetch Data

Enter a ticker and date range you would like to build the model on.  This model takes a a single ticker's data.  Also enter a training size for the proportion of the data you want to include in your training set vs. your test set.

In [23]:
# stock configs
ticker = ['GOOG']
start_date = '2015-04-01'
end_date = '2024-04-05'

# model configs
train_size = 0.8

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 30  # Number of past days we want to use to predict the future.

In [24]:
# Data Fetching
data = fetch_stock_data(ticker, start_date, end_date)[ticker[0]]
data.reset_index(drop=False, inplace=True)
data['Date'] = pd.to_datetime(data['Date']).dt.tz_localize(None)

print(data.shape)
included_days = len(data)
data.head()

(2268, 8)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2015-04-01,27.354897,27.481548,26.901142,27.053724,39261497,0.0,0.0
1,2015-04-02,26.968458,26.968458,26.619267,26.703186,34327989,0.0,0.0
2,2015-04-06,26.538139,26.846792,26.406002,26.764767,26488525,0.0,0.0
3,2015-04-07,26.830338,27.060205,26.726622,26.777481,26057345,0.0,0.0
4,2015-04-08,26.845297,27.118048,26.845297,27.006353,23570536,0.0,0.0


# Explanation:

1. The code starts by calculating the lengths of the short and long moving average windows based on a percentage of the total number of data points in the DataFrame df. The short window is set to 2.5% of the data points, and the long window is set to 5%.
2. A new DataFrame called signals is created with the same index as df. The signals DataFrame will store the trading signals and other relevant information.
3. The 'signal' column in signals is initialized to 0.0, indicating a neutral position.
4. The 'short_ma' column in signals is calculated by applying a rolling mean to the 'Close' prices from the df DataFrame, using the specified short_window as the window size.
5. The 'long_ma' column in signals is calculated by applying a rolling mean to the 'Close' prices from the df DataFrame, using the specified long_window as the window size.
6. The 'signal' column in signals is updated based on the following condition:
    Starting from the index short_window, if the 'short_ma' is greater than the 'long_ma', the 'signal' is set to 1.0 (buy signal), otherwise it is set to 0.0 (neutral signal).


7. The 'positions' column in signals is calculated by taking the difference of the 'signal' column. This represents the actual positions taken based on the trading signals, where a value of 1 indicates a buy position and -1 indicates a sell position.

In [25]:
short_window = int(0.025 * len(data))
long_window = int(0.05 * len(data))

signals = pd.DataFrame(index=data.index)
signals['signal'] = 0.0

signals['short_ma'] = data['Close'].rolling(window=short_window, min_periods=1, center=False).mean()
signals['long_ma'] = data['Close'].rolling(window=long_window, min_periods=1, center=False).mean()

signals['signal'][short_window:] = np.where(signals['short_ma'][short_window:] 
                                            > signals['long_ma'][short_window:], 1.0, 0.0)   
signals['positions'] = signals['signal'].diff()

signals

/var/folders/0c/6nxq1mwn3g99r6b2tcmbw3xw0000gn/T/ipykernel_20490/3475291960.py:10: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




,signal,short_ma,long_ma,positions
0,0.0,27.053724,27.053724,NaN
1,0.0,26.878455,26.878455,0.0
2,0.0,26.840559,26.840559,0.0
3,0.0,26.824790,26.824790,0.0
4,0.0,26.861102,26.861102,0.0
...,...,...,...,...
2263,1.0,144.804464,140.122920,0.0
2264,1.0,145.053392,140.260176,0.0
2265,1.0,145.268928,140.406990,0.0
2266,1.0,145.495713,140.560884,0.0


In [26]:
def buy_stock(real_movement, signal, initial_money=10000, max_buy=1, max_sell=1):
    """
    Function to simulate the buying and selling of stocks based on a trading signal.

    Parameters:
    - real_movement: The actual price movement of the stock in the real world.
    - signal: The trading signal indicating whether to buy (1), sell (-1), or hold (0).
    - initial_money: The initial amount of money available for trading (default: 10000).
    - max_buy: The maximum quantity of shares to buy in a single transaction (default: 1).
    - max_sell: The maximum quantity of shares to sell in a single transaction (default: 1).

    Returns:
    - states_buy: A list of indices indicating the days on which a buy transaction occurred.
    - states_sell: A list of indices indicating the days on which a sell transaction occurred.
    - total_gains: The total gains achieved from the trading strategy.
    - invest: The percentage return on investment.
    - total_shares: The total number of shares held at the end of the trading simulation.
    """

    starting_money = initial_money
    states_sell = []
    states_buy = []
    current_inventory = 0

    def buy(i, initial_money, current_inventory):
        """
        Helper function to execute a buy transaction.

        Parameters:
        - i: The current index (day) of the trading simulation.
        - initial_money: The current amount of money available for trading.
        - current_inventory: The current number of shares held in the inventory.

        Returns:
        - initial_money: The updated amount of money after the buy transaction.
        - current_inventory: The updated number of shares in the inventory after the buy transaction.
        """
        shares = initial_money // real_movement[i]
        if shares < 1:
            print(f"day {i}: total balances {initial_money:.2f}, not enough money to buy a unit price {real_movement[i]:.2f}")
        else:
            buy_units = min(shares, max_buy)
            initial_money -= buy_units * real_movement[i]
            current_inventory += buy_units
            print(f"day {i}: buy {buy_units} units at price {real_movement[i]:.2f}, total balance {initial_money:.2f}")
            states_buy.append(i)
        return initial_money, current_inventory

    # Simulate the trading strategy
    for i in range(real_movement.shape[0] - int(0.025 * len(data))):
        state = signal[i]
        if state == 1:
            initial_money, current_inventory = buy(i, initial_money, current_inventory)
        elif state == -1:
            if current_inventory == 0:
                print(f"day {i}: cannot sell anything, inventory 0")
            else:
                sell_units = min(current_inventory, max_sell)
                current_inventory -= sell_units
                total_sell = sell_units * real_movement[i]
                initial_money += total_sell
                invest = 0
                if states_buy:
                    invest = ((real_movement[i] - real_movement[states_buy[-1]]) / real_movement[states_buy[-1]]) * 100
                print(f"day {i}, sell {sell_units} units at price {total_sell:.2f}, investment {invest:.2f}%, total balance {initial_money:.2f}")
                states_sell.append(i)

    invest = ((initial_money - starting_money) / starting_money) * 100
    total_gains = initial_money - starting_money
    total_shares = current_inventory
    return states_buy, states_sell, total_gains, invest, total_shares


In [27]:
# Execute the trading strategy
states_buy, states_sell, total_gains, invest, total_shares = buy_stock(data.Close, signals['signal'])

day 67: buy 1 units at price 25.84, total balance 9974.16
day 75: buy 1 units at price 33.15, total balance 9941.01
day 76: buy 1 units at price 33.12, total balance 9907.89
day 77: buy 1 units at price 33.10, total balance 9874.79
day 78: buy 1 units at price 32.21, total balance 9842.57
day 79: buy 1 units at price 31.18, total balance 9811.40
day 80: buy 1 units at price 31.36, total balance 9780.03
day 81: buy 1 units at price 31.40, total balance 9748.63
day 82: buy 1 units at price 31.60, total balance 9717.04
day 83: buy 1 units at price 31.63, total balance 9685.41
day 84: buy 1 units at price 31.28, total balance 9654.13
day 85: buy 1 units at price 31.56, total balance 9622.57
day 86: buy 1 units at price 31.46, total balance 9591.10
day 87: buy 1 units at price 32.19, total balance 9558.91
day 88: buy 1 units at price 32.13, total balance 9526.78
day 89: buy 1 units at price 31.76, total balance 9495.02
day 90: buy 1 units at price 31.69, total balance 9463.33
day 91: buy 1 

In [28]:
import plotly.graph_objects as go

starting_money = 10000

close = data['Close']
final_share_price = close[len(close) - 1]  # Final share price
total_portfolio_value = starting_money + total_shares * final_share_price
total_gains = total_portfolio_value - starting_money

fig = go.Figure()

# Candlestick trace
fig.add_trace(go.Candlestick(x=data.index,
                             open=data['Open'],
                             high=data['High'],
                             low=data['Low'],
                             close=data['Close']))

# Buy signals trace
fig.add_trace(go.Scatter(x=[data.index[i] for i in states_buy],
                         y=[close[i] for i in states_buy],
                         mode='markers',
                         name='Buy Signals',
                         marker=dict(symbol='triangle-up', size=10, color='green')))

# Sell signals trace
fig.add_trace(go.Scatter(x=[data.index[i] for i in states_sell],
                         y=[close[i] for i in states_sell],
                         mode='markers',
                         name='Sell Signals',
                         marker=dict(symbol='triangle-down', size=10, color='red')))

# Set layout
fig.update_layout(
    title=f'Total Gains: {total_gains:.2f}, Total Portfolio Value: {total_portfolio_value:.2f}',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark',
    legend=dict(x=0, y=1, orientation='h')
)

fig.show()